In [14]:
import pandas as pd
import numpy as np
import string
import json
import random

In [15]:
## Static Config
locations = ["woreda", "kebele"]
latlong = ["latitude", "longitude"]
charts = ["water service level", "sanitation service level", "hygiene service level"]
default = {
    "water service level" : ["Advanced", "Basic", "Limited", "No Service"],
    "sanitation service level": ["Advanced", "Basic", "Limited", "No Service"],
    "hygiene service level": ["Advanced", "Basic", "Limited", "No Service"]
}

## Table config
table = [
    {
        "name": "water indicators",
        "column": "water service level",
        "indicators": [
            "Main Water Supply for the Facility",
            "Distance of Main Water Supply from Facility",
            "Water is available at the Time of Survey",
        ],
    },
    {
        "name": "sanitation indicators",
        "column": "sanitation service level",
        "indicators": [
            "Type of Toilet for Patients",
            "At least one toilet is usable",
            "Toilets dedicated for staff",
            "Toilets are in sex-separated or gender-neutral rooms",
            "Toilets have menstrual hygiene facilites",
            "Toilets accessible for people with limited mobility",
        ],
    },
    {
        "name": "hygiene indicators",
        "column": "hygiene service level",
        "indicators": [
            "Functional Hand Hygien Facility at Points of Care",
            "Functional handwashing facility at one or more toilets",
        ],
    }
]

In [16]:
df = pd.read_csv('ug_health_data.csv')

In [17]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [18]:
## Fill Column
#default_column = ["Identifier", "Woreda", "Kebele", "Name_of_Health_facility", "Latitude", "Longitude", "Type_of_healthy_facility"]
#for column in df.columns:
#    if column not in default_column and column != "Environmental Cleaning Service Level":
#        df[column] = df[column].apply(lambda x: random.choice(df[column].dropna().unique()))

In [19]:
### Fill Empty Integer Values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num

In [20]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
for strcol in list(df_str):
    df_str[strcol] = df_str[strcol].str.strip()
df[list(df_str)] = df_str

In [21]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [22]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

chart_config = []
loc_config = {}
latlong_config = {}
for key in configs:
    if(configs[key].lower() in charts):
        chart_config.append({'key': key, 'name': configs[key], 'value': default[configs[key].lower()]})
    if(configs[key].lower() in locations):
        loc_config.update({configs[key].lower(): key})
    if(configs[key].lower() in latlong):
        latlong_config.update({configs[key].lower(): key})
        

#### Table
table_config = []
for tb in table:
    column_index = ""
    tb_indicators = []
    for key in configs:
        if tb['column'].lower() == configs[key].lower():
            column_index = key
        if configs[key].lower() in [x.lower() for x in tb['indicators']]:
            tb_indicators.append(key)
    table_config.append({
        "key": column_index,
        "name": tb['name'],
        "indicators": tb_indicators
    })

            
configs['charts'] = chart_config
configs['locations'] = loc_config
configs['latlong'] = latlong_config
configs['table'] = table_config

In [23]:
### Replace Dataset Columns
df.columns = index

In [24]:
configs

{'A': 'Identifier',
 'B': 'Woreda',
 'C': 'Kebele',
 'D': 'Parish',
 'E': 'Village',
 'F': 'Name_of_Health_facility',
 'G': 'Latitude',
 'H': 'Longitude',
 'I': 'Health Facility Level',
 'J': 'Type of Health Facility',
 'K': 'Average Number of Patients Per Day',
 'L': 'Water Service Level',
 'M': 'Main Water Supply for the Facility',
 'N': 'Distance of Main Water Supply from Facility',
 'O': 'Water is available at the Time of Survey',
 'P': 'Sanitation Service Level',
 'Q': 'Is there an improved toilet with permanent superstructure (Wall constructed of Bricks with concrete floor slab and Galvanised iron sheet roofing).',
 'R': 'Type of Toilet for Patients',
 'S': 'At least one toilet is usable',
 'T': 'Toilets dedicated for staff',
 'U': 'Toilets are in sex-separated or gender-neutral rooms',
 'V': 'Toilets have menstrual hygiene facilites',
 'W': 'Toilets accessible for people with limited mobility',
 'X': 'Hygiene Service Level',
 'Y': 'Functional Hand Hygien Facility at Points of Ca

In [26]:
data = list(df.T.to_dict().values())
data[:2]

[{'A': 'xyjv-dksm-buf9',
  'B': 'Agago',
  'C': 'Kotomol',
  'D': 'Apobo',
  'E': 'Kotomor East',
  'F': 'Kotomor  Health Center',
  'G': 2.64305378,
  'H': 33.34821977,
  'I': 'HC III',
  'J': 'Government Health Facility',
  'K': 30,
  'L': 'Basic',
  'M': 'Rain water harvesting',
  'N': 'On premises',
  'O': 'Yes',
  'P': 'Limited',
  'Q': 'Yes',
  'R': 0,
  'S': 'Yes',
  'T': 'No',
  'U': 'Yes',
  'V': 0,
  'W': 0,
  'X': 'No Service',
  'Y': 'Yes',
  'Z': 'No',
  'AA': 'Basic',
  'AB': 'Yes',
  'AC': '',
  'AD': 0,
  'AE': 'Basic',
  'AF': 'Yes',
  'AG': 'Yes'},
 {'A': 'b942-qhhn-uq48',
  'B': 'Agago',
  'C': 'Kuywee',
  'D': 'Paluti',
  'E': 'Kuywee West',
  'F': 'Kuywee Health centre II',
  'G': 3.05747981,
  'H': 33.30269658,
  'I': 'HC II',
  'J': 'Government Health Facility',
  'K': 40,
  'L': 'Limited',
  'M': 'Deep borehole',
  'N': 'On premises',
  'O': '',
  'P': 'Limited',
  'Q': 'Yes',
  'R': 0,
  'S': 'Yes',
  'T': 'No',
  'U': 'Yes',
  'V': 0,
  'W': 0,
  'X': 'Basic',

In [27]:
### to json
df.to_json('../resources/js/data/ug_health_data.json', orient="records", indent=2)

with open('../resources/js/data/ug_health_config.json', 'w') as outfile:
    json.dump(configs, outfile)